<a href="https://colab.research.google.com/github/RodrigoLima82/Tensorflow2.0/blob/master/Aprendizagem_por_Refor%C3%A7o_para_Negocia%C3%A7%C3%A3o_de_A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Etapa 1: Instalação da bibliotecas


In [2]:
!pip install tensorflow-gpu==2.0.0.alpha0

     |████████████████████████████████| 332.1MB 48kB/s 
     |████████████████████████████████| 3.0MB 51.4MB/s 
     |████████████████████████████████| 419kB 50.3MB/s 


In [3]:
!pip install pandas-datareader

## Etapa 2: Importação das bibliotecas

In [4]:
import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or 

In [5]:
tf.__version__

'2.0.0-alpha0'

## Etapa 3: Construção da IA para negociação de ações

In [0]:
class AI_Trader():
  
  def __init__(self, state_size, action_space = 3, model_name = "AITrader"):
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen = 2000)
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    self.model = self.model_builder()
    
  def model_builder(self):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units = 32, activation = "relu", input_dim = self.state_size))
    model.add(tf.keras.layers.Dense(units = 64, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = 128, activation = "relu"))
    model.add(tf.keras.layers.Dense(units = self.action_space, activation = "linear"))
    model.compile(loss = "mse", optimizer = tf.keras.optimizers.Adam(lr = 0.001))
    return model
  
  def trade(self, state):
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay
      

## Etapa 4: Pré-processamento da base de dados

### Definição de funções auxiliares

#### Sigmoid

In [0]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [8]:
sigmoid(0.5)

0.6224593312018546

#### Formatação de preços

In [0]:
def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [10]:
stocks_price_format(100)

'$ 100.000000'

#### Carregador da base de dados

In [0]:
dataset = data_reader.DataReader("^BVSP", data_source = "yahoo")

In [41]:
dataset.tail()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2019-11-19,106950.000000,105367.000000,106269.000000,105618.000000,3610300.0,105618.000000
2019-11-21,107497.000000,105864.000000,105866.000000,107497.000000,5819400.0,107497.000000
2019-11-22,108692.000000,107157.000000,107497.000000,108692.000000,4796100.0,108692.000000
2019-11-25,108915.000000,108080.000000,108692.000000,108424.000000,4512400.0,108424.000000
2019-11-26,108423.929688,106413.929688,108423.929688,107059.398438,0.0,107059.398438


In [37]:
str(dataset.index[0]).split()[0]

'2010-01-04'

In [38]:
dataset.index[-1]

Timestamp('2019-11-26 00:00:00')

In [39]:
dataset['Close']

Date
2010-01-04     70045.000000
2010-01-05     70240.000000
2010-01-06     70729.000000
2010-01-07     70451.000000
2010-01-08     70263.000000
                  ...      
2019-11-19    105618.000000
2019-11-21    107497.000000
2019-11-22    108692.000000
2019-11-25    108424.000000
2019-11-26    107059.398438
Name: Close, Length: 2452, dtype: float64

In [0]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name, data_source = "yahoo")
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

### Criador de estados

In [43]:
0 - 5 + 1

-4

In [44]:
20 - 5 + 1

16

In [45]:
dataset[16:21]

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-27,65663.0,64553.0,65527.0,65070.0,1813600.0,65070.0
2010-01-28,66049.0,64541.0,65071.0,65588.0,1649400.0,65588.0
2010-01-29,66576.0,65140.0,65591.0,65402.0,1823200.0,65402.0
2010-02-01,66763.0,65362.0,65402.0,66572.0,1494400.0,66572.0
2010-02-02,67321.0,66539.0,66589.0,67163.0,1587800.0,67163.0


In [0]:
def state_creator(data, timestep, window_size):
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep + 1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep + 1])
    
  state = []
  for i in range(window_size - 1):
    #state.append(sigmoid(windowed_data[i + 1] - windowed_data[i]))
    state.append(windowed_data[i + 1] - windowed_data[i])
    
  return np.array([state]), windowed_data

### Carregando a base de dados

In [0]:
stock_name = "^BVSP"
data = dataset_loader(stock_name)

In [0]:
s, w = state_creator(data, 0, 5)

In [51]:
s

array([[0.5, 0.5, 0.5, 0.5]])

In [52]:
w

[70045.0, 70045.0, 70045.0, 70045.0, 70045.0]

## Etapa 5: Treinando a IA

### Configuração dos hyper parâmetros

In [0]:
window_size = 10
episodes = 1000
batch_size = 32
data_samples = len(data) - 1

In [54]:
data_samples

2451

### Definição do modelo

In [0]:
trader = AI_Trader(window_size)

In [56]:
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Loop de treinamento

In [59]:
for episode in range(1, episodes + 1):
  print("Episode: {}/{}".format(episode, episodes))
  state = state_creator(data, 0, window_size + 1)
  total_profit = 0
  trader.inventory = []
  for t in tqdm(range(data_samples)):
    action = trader.trade(state)
    next_state = state_creator(data, t + 1, window_size + 1)
    reward = 0
    
    if action == 1: # Comprando uma ação
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
    elif action == 2 and len(trader.inventory) > 0: # Vendendo uma ação  
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price))
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("Total profit: {}".format(total_profit))
      print("########################")
      
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
     
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/2451 [00:00<?, ?it/s]

Episode: 1/1000
AI Trader bought:  $ 70729.000000
AI Trader bought:  $ 70451.000000
AI Trader bought:  $ 70263.000000
AI Trader sold:  $ 70076.000000  Profit: - $ 653.000000
AI Trader bought:  $ 70385.000000
AI Trader sold:  $ 68978.000000  Profit: - $ 1473.000000
AI Trader bought:  $ 69401.000000
AI Trader sold:  $ 68200.000000  Profit: - $ 2063.000000
AI Trader bought:  $ 66270.000000
AI Trader bought:  $ 66220.000000
AI Trader bought:  $ 65524.000000
AI Trader bought:  $ 65070.000000
AI Trader sold:  $ 65588.000000  Profit: - $ 4797.000000
AI Trader sold:  $ 65402.000000  Profit: - $ 3999.000000
AI Trader sold:  $ 66572.000000  Profit: $ 302.000000
AI Trader sold:  $ 67163.000000  Profit: $ 943.000000
AI Trader sold:  $ 67109.000000  Profit: $ 1585.000000
AI Trader sold:  $ 63934.000000  Profit: - $ 1136.000000


  1%|          | 28/2451 [00:02<05:38,  7.15it/s]

AI Trader bought:  $ 66129.000000
AI Trader sold:  $ 65855.000000  Profit: - $ 274.000000


  1%|          | 30/2451 [00:02<06:38,  6.07it/s]

...

AI Trader bought:  $ 54413.000000


 44%|████▍     | 1081/2451 [03:23<04:23,  5.21it/s]

AI Trader sold:  $ 53976.000000  Profit: - $ 437.000000


 45%|████▍     | 1096/2451 [03:25<04:17,  5.27it/s]

KeyboardInterrupt: ignored